# ReGenNexus UAP - Basic Demo

This notebook demonstrates the basic functionality of ReGenNexus UAP (Universal Adapter Protocol) for secure communication between agents, devices, and services.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ReGenNow/ReGenNexus/blob/main/examples/binder/colab_basic_demo.ipynb)

## Setup

Install ReGenNexus from PyPI:

In [ ]:
# Install ReGenNexus from PyPI
!pip install regennexus[full]

# Install additional dependencies for visualization
!pip install matplotlib numpy nest_asyncio

In [ ]:
# Verify installation
import regennexus
print(f"ReGenNexus version: {regennexus.__version__}")

In [ ]:
import asyncio
import json
import time
from IPython.display import display, HTML, clear_output
import matplotlib.pyplot as plt
import numpy as np
import nest_asyncio

# Apply nest_asyncio to allow asyncio to work in Jupyter
nest_asyncio.apply()

In [ ]:
# Import ReGenNexus modules
from regennexus.core.protocol import RegenNexus, get_instance, get_registry
from regennexus.core.message import Message, Intent
from regennexus.core.entity import Entity
from regennexus.security.security import SecurityManager

## Create Demo Environment

Let's set up a simple demo environment with message passing through ReGenNexus:

In [ ]:
# Create the protocol instance
protocol = None
registry = None
messages_log = []

async def setup_demo():
    """Set up the demo environment."""
    global protocol, registry
    
    print("Setting up demo environment...")
    
    # Create protocol instance
    protocol = RegenNexus()
    await protocol.start()
    
    # Get registry
    registry = get_registry()
    
    # Register demo entities using Entity objects
    agent1 = Entity(
        entity_id="agent_1",
        entity_type="agent",
        capabilities=["chat", "task_execution"]
    )
    await registry.register(agent1)
    
    agent2 = Entity(
        entity_id="agent_2",
        entity_type="agent",
        capabilities=["chat", "data_processing"]
    )
    await registry.register(agent2)
    
    sensor1 = Entity(
        entity_id="sensor_1",
        entity_type="device",
        capabilities=["temperature", "humidity"]
    )
    await registry.register(sensor1)
    
    print("Demo environment ready!")
    print("\nRegistered entities:")
    entities = await registry.list_entities()
    for entity in entities:
        print(f"  - {entity['id']}: {entity['entity_type']} ({entity['capabilities']})")

# Run setup
asyncio.get_event_loop().run_until_complete(setup_demo())

## Message Passing Demo

Let's send some messages between entities:

In [ ]:
async def send_demo_messages():
    """Send demo messages between entities."""
    global messages_log
    
    print("\nSending demo messages...\n")
    
    # Message 1: Agent 1 -> Agent 2 (Query)
    msg1 = Message(
        sender_id="agent_1",
        recipient_id="agent_2",
        intent=Intent.QUERY,
        content={"question": "What is your status?"}
    )
    await registry.route_message(msg1)
    print(f"[Agent 1 -> Agent 2] Query: What is your status?")
    messages_log.append(("agent_1", "agent_2", "Query: What is your status?"))
    
    await asyncio.sleep(0.5)
    
    # Message 2: Agent 2 -> Agent 1 (Response)
    msg2 = Message(
        sender_id="agent_2",
        recipient_id="agent_1",
        intent=Intent.RESPONSE,
        content={"status": "operational", "load": 0.45}
    )
    await registry.route_message(msg2)
    print(f"[Agent 2 -> Agent 1] Response: status=operational, load=45%")
    messages_log.append(("agent_2", "agent_1", "Response: operational, 45% load"))
    
    await asyncio.sleep(0.5)
    
    # Message 3: Agent 1 -> Sensor (Command to read sensors)
    msg3 = Message(
        sender_id="agent_1",
        recipient_id="sensor_1",
        intent=Intent.COMMAND,
        content={"action": "read", "sensors": ["temperature", "humidity"]}
    )
    await registry.route_message(msg3)
    print(f"[Agent 1 -> Sensor 1] Command: read temperature, humidity")
    messages_log.append(("agent_1", "sensor_1", "Command: read sensors"))
    
    await asyncio.sleep(0.5)
    
    # Message 4: Sensor -> Agent 1 (Sensor Data)
    msg4 = Message(
        sender_id="sensor_1",
        recipient_id="agent_1",
        intent=Intent.SENSOR_DATA,
        content={"temperature": 22.5, "humidity": 65.2, "unit": "celsius"}
    )
    await registry.route_message(msg4)
    print(f"[Sensor 1 -> Agent 1] Sensor Data: temp=22.5C, humidity=65.2%")
    messages_log.append(("sensor_1", "agent_1", "Data: 22.5C, 65.2%"))
    
    print("\nDemo messages sent!")

# Run message demo
asyncio.get_event_loop().run_until_complete(send_demo_messages())

## Visualize Message Flow

Let's create a visualization of the message flow:

In [ ]:
def visualize_message_flow():
    """Create a visualization of message flow between entities."""
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Entity positions
    positions = {
        "agent_1": (0.2, 0.7),
        "agent_2": (0.8, 0.7),
        "sensor_1": (0.5, 0.3)
    }
    
    colors = {
        "agent_1": "#3498db",
        "agent_2": "#2ecc71",
        "sensor_1": "#e74c3c"
    }
    
    # Draw entities
    for entity_id, pos in positions.items():
        circle = plt.Circle(pos, 0.08, color=colors[entity_id], alpha=0.8)
        ax.add_patch(circle)
        ax.text(pos[0], pos[1]-0.15, entity_id.replace("_", " ").title(), 
                ha='center', fontsize=11, fontweight='bold')
    
    # Draw message arrows
    arrow_colors = ['#3498db', '#2ecc71', '#3498db', '#e74c3c']
    
    for i, (sender, recipient, msg) in enumerate(messages_log):
        start = positions[sender]
        end = positions[recipient]
        
        # Calculate arrow offset for multiple messages
        offset = 0.02 * (i % 2) * (1 if i < 2 else -1)
        
        # Draw arrow
        ax.annotate("", 
                    xy=(end[0], end[1] + offset), 
                    xytext=(start[0], start[1] + offset),
                    arrowprops=dict(arrowstyle="->", color=arrow_colors[i], lw=2))
        
        # Add message label
        mid_x = (start[0] + end[0]) / 2
        mid_y = (start[1] + end[1]) / 2 + offset + 0.05
        ax.text(mid_x, mid_y, f"{i+1}. {msg[:30]}.." if len(msg) > 30 else f"{i+1}. {msg}", 
                ha='center', fontsize=9, style='italic',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # Set limits and styling
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.set_title("ReGenNexus UAP - Message Flow Visualization", fontsize=14, fontweight='bold')
    ax.axis('off')
    
    # Add legend
    legend_elements = [
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='#3498db', markersize=15, label='Agent 1'),
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='#2ecc71', markersize=15, label='Agent 2'),
        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='#e74c3c', markersize=15, label='Sensor 1')
    ]
    ax.legend(handles=legend_elements, loc='upper left')
    
    plt.tight_layout()
    plt.show()

# Visualize
visualize_message_flow()

## Security Features Demo

ReGenNexus includes built-in security features:

In [ ]:
async def demo_security():
    """Demonstrate security features."""
    print("Security Features Demo")
    print("=" * 40)
    
    # Create two security managers (simulating two parties)
    alice = SecurityManager()
    bob = SecurityManager()
    
    # 1. Show public keys
    alice_pub = alice.get_public_key()
    bob_pub = bob.get_public_key()
    print(f"\n1. Key Generation:")
    print(f"   Alice's public key: {alice_pub[:30].hex()}...")
    print(f"   Bob's public key: {bob_pub[:30].hex()}...")
    print(f"   ECDH supported: {alice.supports_ecdh()}")
    
    # 2. Encrypt message from Alice to Bob
    test_data = b"Hello Bob, this is a secret message from Alice!"
    
    encrypted = await alice.encrypt_with_best_available(test_data, bob_pub)
    print(f"\n2. Encryption (Alice -> Bob):")
    print(f"   Original: {test_data.decode()}")
    print(f"   Encrypted: {encrypted[:60]}...")
    
    # 3. Bob decrypts the message
    decrypted = await bob.decrypt_message(encrypted)
    print(f"\n3. Decryption (Bob receives):")
    print(f"   Decrypted: {decrypted.decode()}")
    print(f"   Match: {test_data == decrypted}")
    
    # 4. Sign and verify
    signature = await alice.sign_data(test_data)
    print(f"\n4. Digital Signature:")
    print(f"   Signature: {signature[:30].hex()}...")
    
    is_valid = await alice.verify_signature(test_data, signature, alice_pub)
    print(f"\n5. Signature Verification:")
    print(f"   Valid: {is_valid}")
    
    print(f"\n6. Security Level: {alice.security_level}")

# Run security demo
asyncio.get_event_loop().run_until_complete(demo_security())

## MCP Tools Demo

ReGenNexus includes MCP (Model Context Protocol) support for AI integration:

In [ ]:
from regennexus.bridges.mcp_bridge import create_hardware_mcp_server, MCPServer

# Create MCP server with hardware tools
mcp_server = create_hardware_mcp_server()

print("MCP Server Tools:")
print("=" * 40)
for name, tool in mcp_server._tools.items():
    print(f"\n{name}:")
    print(f"  Description: {tool.description}")
    print(f"  Parameters: {list(tool.input_schema.get('properties', {}).keys())}")

## Clean Up

In [ ]:
async def cleanup():
    """Clean up resources."""
    global protocol
    if protocol:
        await protocol.stop()
        print("Demo environment cleaned up.")

asyncio.get_event_loop().run_until_complete(cleanup())

## Conclusion

This notebook demonstrated:

1. **Entity Registration** - Registering agents and devices
2. **Message Passing** - Sending messages with different intents
3. **Visualization** - Visualizing message flow
4. **Security** - Token generation, validation, and encryption
5. **MCP Integration** - Hardware control tools for AI agents

### Next Steps

- Check out the [Security Demo](https://colab.research.google.com/github/ReGenNow/ReGenNexus/blob/main/examples/binder/colab_security_demo.ipynb)
- Visit the [GitHub Repository](https://github.com/ReGenNow/ReGenNexus) for more examples
- Install via PyPI: `pip install regennexus[full]`

**ReGenNexus UAP** - Connect Everything, Securely.